In [1]:
# Import Library Dependencies
from sqlalchemy import create_engine, engine, inspect
from config import user, password

import datetime as dt
import numpy as np
import pandas as pd
import os

In [2]:
# Establish CSV Data Path Reference
csvName = '2019-06-22-StudentMobility.csv'       # <<< Change The CSV Name And Path to Match Your Computer
filePath = os.path.join('Datasets', csvName)     # <<< And This -- My Data Is Nested Under An Additional Folder
filePath

'Datasets\\2019-06-22-StudentMobility.csv'

In [3]:
# Define & Rearrange Desired Column Headers
import_headers = [
    'NID',  # Student Id
    'GR',   # Grade`
    'SC',   # School Code
    'PR',   # School Program
    'ED',   # Entry Date
    'LD',   # Leave Date
    'ER'    # Exit Reason**
]

In [4]:
# Read CSV Data Into DataFrame
master_df = pd.read_csv(filePath, low_memory=False)[import_headers]
master_df.head(10)

,NID,GR,SC,PR,ED,LD,ER
0,300265.0,12.0,27.0,NaN,8/9/2017,5/24/2018,230.0
1,300456.0,12.0,28.0,NaN,8/9/2017,5/24/2018,230.0
2,301428.0,12.0,23.0,NaN,8/9/2017,5/24/2018,230.0
3,301661.0,12.0,20.0,NaN,8/9/2017,9/6/2017,160.0
4,301759.0,12.0,61.0,I,8/9/2017,12/22/2017,167.0
5,303104.0,12.0,68.0,NaN,8/9/2017,3/9/2018,165.0
6,303301.0,12.0,22.0,NaN,8/9/2017,5/24/2018,167.0
7,303962.0,12.0,27.0,NaN,8/9/2017,5/24/2018,230.0
8,304281.0,12.0,28.0,NaN,8/9/2017,5/24/2018,230.0
9,304396.0,12.0,22.0,NaN,8/9/2017,11/13/2017,440.0


In [5]:
# Rename Column Headers to Match Database Field Names
update_headers = {
    'NID': 'student_id',
    'GR': 'student_grade_level',
    'SC': 'student_school_id',
    'PR': 'student_program_id',
    'ED': 'student_entry_date',
    'LD': 'student_leave_date',
    'ER': 'student_exit_reason'
}

master_df = master_df.rename(columns=update_headers)
master_df.head()

,student_id,student_grade_level,student_school_id,student_program_id,student_entry_date,student_leave_date,student_exit_reason
0,300265.0,12.0,27.0,NaN,8/9/2017,5/24/2018,230.0
1,300456.0,12.0,28.0,NaN,8/9/2017,5/24/2018,230.0
2,301428.0,12.0,23.0,NaN,8/9/2017,5/24/2018,230.0
3,301661.0,12.0,20.0,NaN,8/9/2017,9/6/2017,160.0
4,301759.0,12.0,61.0,I,8/9/2017,12/22/2017,167.0


In [6]:
# Preview Dataset -- Check for Missing Data (Data Value Should Be Equal)
master_df.count()

student_id             62774
student_grade_level    62774
student_school_id      62774
student_program_id      7340
student_entry_date     62774
student_leave_date     21789
student_exit_reason    21787
dtype: int64

In [7]:
# Reduce/Truncate Dataset
# Truncate Dataset to Records Containing an Exit Reason
reduced_df = master_df.loc[master_df['student_exit_reason'].isnull()==False]

# Fill Remaining Missing Data In Program_Id Column With String Values For Later Data Conversion
reduced_df['student_program_id'] = reduced_df['student_program_id'].fillna('N/a')
reduced_df.count()

C:\Users\Ryan Tamashiro\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


student_id             21787
student_grade_level    21787
student_school_id      21787
student_program_id     21787
student_entry_date     21787
student_leave_date     21787
student_exit_reason    21787
dtype: int64

In [8]:
# Specify Required Column Data Types
fmt_dates = [
    'student_entry_date', 
    'student_leave_date'
]

dtype = {
    'student_id':int,
    'student_grade_level':int,
    'student_school_id':int,
    'student_program_id':str,
    'student_exit_reason':int
}

# Preview Current Column Data Types
reduced_df.dtypes

student_id             float64
student_grade_level    float64
student_school_id      float64
student_program_id      object
student_entry_date      object
student_leave_date      object
student_exit_reason    float64
dtype: object

In [9]:
# Data Conversion on DataFrame Colums to Match Those Required By MySQL Database
converted_df = reduced_df.astype(dtype, errors='ignore')
converted_df[fmt_dates] = converted_df[fmt_dates].apply(pd.to_datetime, errors='ignore')
converted_df.dtypes

student_id                      int32
student_grade_level             int32
student_school_id               int32
student_program_id             object
student_entry_date     datetime64[ns]
student_leave_date     datetime64[ns]
student_exit_reason             int32
dtype: object

In [10]:
# Create DataFrames For Initial MySQL Database Upload
# <Create> Students DataFrame Upload: 
student_df = pd.DataFrame()
student_df['student_id'] = converted_df['student_id'].unique()
student_df['student_fname'] = 'Protected'
student_df['student_lname'] = 'Protected'
student_df['updated_on'] = dt.datetime.today()
student_df['updated_by'] = 'Ryan Tamashiro'

student_df.head()

,student_id,student_fname,student_lname,updated_on,updated_by
0,300265,Protected,Protected,2019-06-26 20:26:52.010350,Ryan Tamashiro
1,300456,Protected,Protected,2019-06-26 20:26:52.010350,Ryan Tamashiro
2,301428,Protected,Protected,2019-06-26 20:26:52.010350,Ryan Tamashiro
3,301661,Protected,Protected,2019-06-26 20:26:52.010350,Ryan Tamashiro
4,301759,Protected,Protected,2019-06-26 20:26:52.010350,Ryan Tamashiro


In [11]:
# <Create> Schools DataFrame Upload:
schools_df = pd.DataFrame()
schools_df['school_id'] = converted_df['student_school_id'].unique()
schools_df['school_name'] = 'Withheld'
schools_df['school_address'] = 'Withheld'
schools_df['school_district'] = 'UCI DATA'

schools_df.head()

,school_id,school_name,school_address,school_district
0,27,Withheld,Withheld,UCI DATA
1,28,Withheld,Withheld,UCI DATA
2,23,Withheld,Withheld,UCI DATA
3,20,Withheld,Withheld,UCI DATA
4,61,Withheld,Withheld,UCI DATA


In [12]:
# <Create> Student Records DataFrame Upload

upload_headers = [
    'student_id',
    'student_school_id',
    'student_entry_date',
    'student_grade_level',
    'student_program_id',
    'student_leave_date',
    'student_exit_reason'
]

student_records_df = converted_df[upload_headers]
student_records_df['gifted_student'] = True
student_records_df['disability_student'] = False
student_records_df['esl_student'] = False
# student_records_df['student_school_district'] = 'UCI_DATA'

student_records_df.head()

C:\Users\Ryan Tamashiro\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Ryan Tamashiro\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\Ryan Tamashiro\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

,student_id,student_school_id,student_entry_date,student_grade_level,student_program_id,student_leave_date,student_exit_reason,gifted_student,disability_student,esl_student
0,300265,27,2017-08-09,12,N/a,2018-05-24,230,True,False,False
1,300456,28,2017-08-09,12,N/a,2018-05-24,230,True,False,False
2,301428,23,2017-08-09,12,N/a,2018-05-24,230,True,False,False
3,301661,20,2017-08-09,12,N/a,2017-09-06,160,True,False,False
4,301759,61,2017-08-09,12,I,2017-12-22,167,True,False,False


In [13]:
# <Create> School Programs DataFrame Upload
school_programs_df = pd.DataFrame()
school_programs_df['program_id'] = converted_df['student_program_id'].unique()
school_programs_df['program_name'] = 'Withheld/Protected'
school_programs_df['program_description'] = 'School/State Sponsored'

school_programs_df

,program_id,program_name,program_description
0,N/a,Withheld/Protected,School/State Sponsored
1,I,Withheld/Protected,School/State Sponsored
2,S,Withheld/Protected,School/State Sponsored
3,C,Withheld/Protected,School/State Sponsored
4,H,Withheld/Protected,School/State Sponsored
5,N,Withheld/Protected,School/State Sponsored


In [14]:
# Upload Student Profile Data to 'students' Table in student_mobility database
MySQL_DB_Connection = f'{user}:{password}@localhost/student_mobility'
    
engine = create_engine(f'mysql://{MySQL_DB_Connection}', echo=True)
engine.table_names()

2019-06-26 20:27:00,390 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'sql_mode'
2019-06-26 20:27:00,390 INFO sqlalchemy.engine.base.Engine ()
2019-06-26 20:27:00,393 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'lower_case_table_names'
2019-06-26 20:27:00,393 INFO sqlalchemy.engine.base.Engine ()
2019-06-26 20:27:00,396 INFO sqlalchemy.engine.base.Engine SELECT DATABASE()
2019-06-26 20:27:00,397 INFO sqlalchemy.engine.base.Engine ()
2019-06-26 20:27:00,398 INFO sqlalchemy.engine.base.Engine show collation where `Charset` = 'utf8mb4' and `Collation` = 'utf8mb4_bin'
2019-06-26 20:27:00,399 INFO sqlalchemy.engine.base.Engine ()
2019-06-26 20:27:00,401 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS CHAR(60)) AS anon_1
2019-06-26 20:27:00,402 INFO sqlalchemy.engine.base.Engine ()
2019-06-26 20:27:00,404 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS CHAR(60)) AS anon_1
2019-06-26 20:27:00,405 INFO sqlalchemy.engine.base.E

['cities', 'school_programs', 'schools', 'student_records', 'students']

In [15]:
# [Upload #1] Upload Student Table Data
student_df.to_sql(
    name='students', 
    con=engine, 
    if_exists='append',
    index=False
)

2019-06-26 20:27:01,515 INFO sqlalchemy.engine.base.Engine DESCRIBE `students`
2019-06-26 20:27:01,516 INFO sqlalchemy.engine.base.Engine ()
2019-06-26 20:27:01,523 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-06-26 20:27:01,581 INFO sqlalchemy.engine.base.Engine INSERT INTO students (student_id, student_fname, student_lname, updated_on, updated_by) VALUES (%s, %s, %s, %s, %s)
2019-06-26 20:27:01,581 INFO sqlalchemy.engine.base.Engine ((300265, 'Protected', 'Protected', datetime.datetime(2019, 6, 26, 20, 26, 52, 10350), 'Ryan Tamashiro'), (300456, 'Protected', 'Protected', datetime.datetime(2019, 6, 26, 20, 26, 52, 10350), 'Ryan Tamashiro'), (301428, 'Protected', 'Protected', datetime.datetime(2019, 6, 26, 20, 26, 52, 10350), 'Ryan Tamashiro'), (301661, 'Protected', 'Protected', datetime.datetime(2019, 6, 26, 20, 26, 52, 10350), 'Ryan Tamashiro'), (301759, 'Protected', 'Protected', datetime.datetime(2019, 6, 26, 20, 26, 52, 10350), 'Ryan Tamashiro'), (303104, 'Protected', '

In [16]:
# [Upload #2] Upload School Table Data
schools_df.to_sql(
    name='schools',
    con=engine,
    if_exists='append',
    index=False
)

2019-06-26 20:27:02,469 INFO sqlalchemy.engine.base.Engine DESCRIBE `schools`
2019-06-26 20:27:02,470 INFO sqlalchemy.engine.base.Engine ()
2019-06-26 20:27:02,474 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-06-26 20:27:02,475 INFO sqlalchemy.engine.base.Engine INSERT INTO schools (school_id, school_name, school_address, school_district) VALUES (%s, %s, %s, %s)
2019-06-26 20:27:02,476 INFO sqlalchemy.engine.base.Engine ((27, 'Withheld', 'Withheld', 'UCI DATA'), (28, 'Withheld', 'Withheld', 'UCI DATA'), (23, 'Withheld', 'Withheld', 'UCI DATA'), (20, 'Withheld', 'Withheld', 'UCI DATA'), (61, 'Withheld', 'Withheld', 'UCI DATA'), (68, 'Withheld', 'Withheld', 'UCI DATA'), (22, 'Withheld', 'Withheld', 'UCI DATA'), (24, 'Withheld', 'Withheld', 'UCI DATA')  ... displaying 10 of 30 total bound parameter sets ...  (472, 'Withheld', 'Withheld', 'UCI DATA'), (999, 'Withheld', 'Withheld', 'UCI DATA'))
2019-06-26 20:27:02,478 INFO sqlalchemy.engine.base.Engine COMMIT


In [17]:
# [Upload #3] Upload School Program Data 
school_programs_df.iloc[1:].to_sql(
    name='school_programs',
    con=engine,
    if_exists='append',
    index=False
)

2019-06-26 20:27:03,008 INFO sqlalchemy.engine.base.Engine DESCRIBE `school_programs`
2019-06-26 20:27:03,010 INFO sqlalchemy.engine.base.Engine ()
2019-06-26 20:27:03,014 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-06-26 20:27:03,015 INFO sqlalchemy.engine.base.Engine INSERT INTO school_programs (program_id, program_name, program_description) VALUES (%s, %s, %s)
2019-06-26 20:27:03,015 INFO sqlalchemy.engine.base.Engine (('I', 'Withheld/Protected', 'School/State Sponsored'), ('S', 'Withheld/Protected', 'School/State Sponsored'), ('C', 'Withheld/Protected', 'School/State Sponsored'), ('H', 'Withheld/Protected', 'School/State Sponsored'), ('N', 'Withheld/Protected', 'School/State Sponsored'))
2019-06-26 20:27:03,017 INFO sqlalchemy.engine.base.Engine COMMIT


In [18]:
# [Upload #4] Upload Student Record Data
student_records_df.to_sql(
    name='student_records',
    con=engine,
    if_exists='append',
    index=False
)

2019-06-26 20:27:03,752 INFO sqlalchemy.engine.base.Engine DESCRIBE `student_records`
2019-06-26 20:27:03,754 INFO sqlalchemy.engine.base.Engine ()
2019-06-26 20:27:03,764 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-06-26 20:27:03,912 INFO sqlalchemy.engine.base.Engine INSERT INTO student_records (student_id, student_school_id, student_entry_date, student_grade_level, student_program_id, student_leave_date, student_exit_reason, gifted_student, disability_student, esl_student) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
2019-06-26 20:27:03,912 INFO sqlalchemy.engine.base.Engine ((300265, 27, datetime.datetime(2017, 8, 9, 0, 0), 12, 'N/a', datetime.datetime(2018, 5, 24, 0, 0), 230, 1, 0, 0), (300456, 28, datetime.datetime(2017, 8, 9, 0, 0), 12, 'N/a', datetime.datetime(2018, 5, 24, 0, 0), 230, 1, 0, 0), (301428, 23, datetime.datetime(2017, 8, 9, 0, 0), 12, 'N/a', datetime.datetime(2018, 5, 24, 0, 0), 230, 1, 0, 0), (301661, 20, datetime.datetime(2017, 8, 9, 0, 0), 12, 'N